In [1]:

!pip install numpy pandas matplotlib neurokit2



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import pandas as pd
import numpy as np

# Load fiducial points detected earlier
fiducials = pd.read_csv("fiducial_points.csv")
print("Loaded Fiducial Points:")
display(fiducials.head())


Loaded Fiducial Points:


,P_peak,Q_peak,R_peak,S_peak,T_peak
0,101.0,111.0,115.0,127.0,143.0
1,194.0,204.0,208.0,215.0,236.0
2,289.0,297.0,300.0,305.0,328.0
3,380.0,388.0,393.0,402.0,422.0
4,476.0,482.0,487.0,492.0,516.0


In [3]:

import wfdb
import neurokit2 as nk

# Load ECG signal for the same record used in fiducial detection
data_folder = "Dataset/ptbxl_data"
import os
sample_path = "records100/00000/00001_lr"  # Update if needed
record_path = os.path.join(data_folder, sample_path)
signal, _ = wfdb.rdsamp(record_path)
fs = 100  # Sampling frequency

# Use Lead II
ecg_cleaned = nk.ecg_clean(signal[:, 1], sampling_rate=fs)


In [8]:

def time_diff(t1, t2, fs):
    return (t2 - t1) / fs

def get_baseline(signal, idx, window=10):
    start = max(0, int(idx) - window)
    return np.mean(signal[start:int(idx)])

def compute_statistics(values):
    return {
        "mean": np.nanmean(values),
        "std": np.nanstd(values),
        "skew": pd.Series(values).skew()
    }


In [12]:

features = []

P_peaks = fiducials["P_peak"].dropna().astype(int).values
Q_peaks = fiducials["Q_peak"].dropna().astype(int).values
R_peaks = fiducials["R_peak"].dropna().astype(int).values
S_peaks = fiducials["S_peak"].dropna().astype(int).values
T_peaks = fiducials["T_peak"].dropna().astype(int).values

for i in range(min(len(P_peaks), len(Q_peaks), len(R_peaks)-1, len(S_peaks), len(T_peaks))):
    P_p = P_peaks[i]
    Q_p = Q_peaks[i]
    R_p = R_peaks[i]
    S_p = S_peaks[i]
    T_p = T_peaks[i]
    
    # Local baseline
    baseline = get_baseline(ecg_cleaned, P_p)
    
    # --- Time Intervals ---
    PR = time_diff(P_p, Q_p, fs)
    QRS = time_diff(Q_p, S_p, fs)
    QT = time_diff(Q_p, T_p, fs)
    RR = time_diff(R_p, R_peaks[i+1], fs)
    QTc = QT / np.sqrt(RR)
    
    # --- Amplitudes ---
    P_amp = ecg_cleaned[P_p] - baseline
    Q_depth = abs(ecg_cleaned[Q_p] - baseline)
    R_amp = ecg_cleaned[R_p] - baseline
    ST = ecg_cleaned[S_p + int(0.06 * fs)] - baseline if S_p + int(0.06 * fs) < len(ecg_cleaned) else np.nan
    T_amp = ecg_cleaned[T_p] - baseline
    T_area = np.trapz(ecg_cleaned[T_p:T_p+int(0.2*fs)] - baseline) if T_p+int(0.2*fs) < len(ecg_cleaned) else np.nan
    
    # --- Derived Ratios ---
    RS_ratio = R_amp / (abs(ecg_cleaned[S_p] - baseline) if abs(ecg_cleaned[S_p] - baseline) > 0 else np.nan)
    ST_slope = (ST - (ecg_cleaned[S_p] - baseline)) / 0.06 if not np.isnan(ST) else np.nan
    
    features.append([PR, QRS, QT, QTc, RR, P_amp, Q_depth, R_amp, ST, T_amp, T_area, RS_ratio, ST_slope])

# Build feature dataframe
df_features = pd.DataFrame(features, columns=[
    "PR", "QRS", "QT", "QTc", "RR", "P_amp", "Q_depth", "R_amp", 
    "ST", "T_amp", "T_area", "RS_ratio", "ST_slope"
])

print("Extracted ECG Features:")
display(df_features.head())

# Save features
df_features.to_csv("extracted_ecg_features.csv", index=False)
print("✅ Features saved to extracted_ecg_features.csv")


Extracted ECG Features:


C:\Users\Hajel\AppData\Local\Temp\ipykernel_24368\2033956393.py:32: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  T_area = np.trapz(ecg_cleaned[T_p:T_p+int(0.2*fs)] - baseline) if T_p+int(0.2*fs) < len(ecg_cleaned) else np.nan


,PR,QRS,QT,QTc,RR,P_amp,Q_depth,R_amp,ST,T_amp,T_area,RS_ratio,ST_slope
0,0.10,0.16,0.32,0.331825,0.93,0.068301,0.050799,0.298095,-0.020410,0.190460,0.697616,8.016340,0.279604
1,0.10,0.11,0.32,0.333623,0.92,0.082754,0.037023,0.309971,-0.001647,0.199120,0.561572,7.845904,0.631002
2,0.08,0.08,0.31,0.321455,0.93,0.081022,0.049445,0.299094,-0.017409,0.193233,0.346693,4.990207,0.708785
3,0.08,0.14,0.34,0.350683,0.94,0.058688,0.068742,0.295182,-0.045754,0.175926,0.246123,5.191181,0.185145
4,0.06,0.10,0.34,0.347011,0.96,0.078565,0.068778,0.260154,-0.016489,0.182138,0.085075,4.551379,0.677833


✅ Features saved to extracted_ecg_features.csv
